<a href="https://colab.research.google.com/github/1900690/draw-image/blob/main/draw_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from io import BytesIO

def generate_random_parameters():
    """ ランダムなパラメータを生成 """
    white_radius = random.uniform(0.4, 0.7)
    green_radius = random.uniform(0.01, 0.5)
    brown_size = random.uniform(0.01, 0.3)
    brown_angle = random.uniform(0, np.pi)
    brown_x = np.cos(brown_angle) * white_radius
    brown_y = np.sin(brown_angle) * white_radius
    angle_rand=random.uniform(0, np.pi)
    return white_radius, green_radius, brown_size, brown_x, brown_y,angle_rand

def create_figure():
    """ 図を作成し、画像としてメモリに保存 """
    white_radius, green_radius, brown_size, brown_x, brown_y ,angle_rand= generate_random_parameters()
    angles = [np.pi / 2+angle_rand, -np.pi / 6+angle_rand, -5 * np.pi / 6+angle_rand]
    green_positions = [(np.cos(angle) * white_radius, np.sin(angle) * white_radius) for angle in angles]
    brown_positions = [(np.cos(angle) * white_radius, np.sin(angle) * white_radius) for angle in angles]

    fig, ax = plt.subplots()
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_aspect('equal')
    #背景を決める
    ax.set_facecolor('#dcdcda')
    ax.axis('off')

    # 緑の円（3つ）
    for x, y in green_positions:
        ax.add_patch(plt.Circle((x, y), green_radius, color='#85a514', zorder=4))

    # 茶色の楕円（3つ）
    for x, y in green_positions:
        ax.add_patch(plt.Circle((x,y), brown_size, color='#a13f2d', zorder=5))

    # 画像をメモリに保存
    buf = BytesIO()
    fig.savefig(buf, format='png', dpi=300)
    plt.close(fig)
    buf.seek(0)
    return buf

def apply_circle_mask(image_buffer, output_file):
    """ 円形マスクを適用し、結果を保存 """
    # メモリから画像を読み込み
    image_array = np.frombuffer(image_buffer.getvalue(), dtype=np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    height, width = img.shape[:2]
    mask = np.zeros_like(img, dtype=np.uint8)
    circle_mask = cv2.circle(mask, (width // 2, height // 2), 300, (255, 255, 255), thickness=-1)
    masked_img = cv2.bitwise_and(img, circle_mask)
    #cv2.imwrite(output_file, masked_img)

    # 画像の中心部分を切り取る
    center_x, center_y = width // 2, height // 2
    crop_size = 300  # 切り取りサイズ
    cropped_img = masked_img[center_y - crop_size:center_y + crop_size, center_x - crop_size:center_x + crop_size]

    cv2.imwrite(output_file, cropped_img)

def main(output_folder='generated_images', num_images=5):
    """ 指定回数繰り返し、フォルダ内に別々の画像を保存 """
    os.makedirs(output_folder, exist_ok=True)
    for i in range(num_images):
        image_buffer = create_figure()
        output_file = os.path.join(output_folder, f'output_{i+1}.jpg')
        apply_circle_mask(image_buffer, output_file)
    print(f"{num_images} images saved in '{output_folder}'")

if __name__ == "__main__":
    main()

5 images saved in 'generated_images'
